In [ ]:
import src.utilities.utils as utils 
test_data = utils.read_jsonl('data/snli_1.0/snli_1.0_test.jsonl')

In [ ]:
def convert_list(lst):
    new_lst = []
    for each in lst:
        new_lst.append(utils.convert_dict[each])
    return new_lst

def exists_intersection(found_first_list, found_second_list):
    lst1 = convert_list(found_first_list)
    lst2 = convert_list(found_second_list)
    return len([value for value in lst1 if value in lst2]) > 0

words_for_numbers = ['a', 'an', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten','1','2','3','4','5','6','7',
                    '8','9','10']
uncertain = ['group', 'some', 'most']

In [ ]:
# FIND potential matching number without addition from entailment pairs

count = 0
for each in test_data:
    found_first, found_second = False, False
    found_first_list, found_second_list = [], []
    for num_word in words_for_numbers:
        if num_word in each['sentence1'].lower().split():
            found_first = True
            found_first_list.append(num_word)
        if num_word in each['sentence2'].lower().split(): 
            found_second = True
            found_second_list.append(num_word)
    for num_word in uncertain:
        if num_word in each['sentence1'].lower().split():
            found_first = False
        if num_word in each['sentence2'].lower().split(): 
            found_second = False
            
    if found_first and found_second and exists_intersection(found_first_list, found_second_list) and 'entailment' == each['gold_label']:
        print(found_first_list, found_second_list, each['gold_label'])
        count += 1
        print(each['sentence1'])
        print(each['sentence2'])
        print(count) 

In [ ]:
# FIND potential addition from entailment pairs

count = 0
for each in test_data:
    found_first, found_second = False, False
    found_first_list, found_second_list = [], []
    for num_word in words_for_numbers:
        if num_word in each['sentence1'].lower().split():
            found_first = True
            found_first_list.append(num_word)
        if num_word in each['sentence2'].lower().split(): 
            found_second = True
            found_second_list.append(num_word)
    for num_word in uncertain:
        if num_word in each['sentence1'].lower().split():
            found_first = False
        if num_word in each['sentence2'].lower().split(): 
            found_second = False
            
    if found_first and found_second and not exists_intersection(found_first_list, found_second_list) and 'entailment' == each['gold_label']:
        print(found_first_list, found_second_list, each['gold_label'])
        count += 1
        print(each['sentence1'])
        print(each['sentence2'])
        print(count) 